In [2]:
from tqdm import tqdm

from openff.recharge.conformers import ConformerGenerator, ConformerSettings
from openff.recharge.esp import ESPSettings
#from openff.recharge.esp.psi4 import Psi4ESPGenerator
from openff_psi4_debug import Psi4ESPGenerator
from openff.recharge.esp.storage import MoleculeESPRecord, MoleculeESPStore
from openff.recharge.grids import LatticeGridSettings
from openff.recharge.utilities.molecule import smiles_to_molecule
from openff.units.elements import SYMBOLS
from rdkit.Chem import AllChem
from openff.toolkit import Molecule
from rdkit.Chem.rdForceFieldHelpers import MMFFOptimizeMolecule

import os
import numpy as np
import random

In [3]:
#load our molecule to create the conformers

molecule = smiles_to_molecule("OCC(O)CO")
rdmol = molecule.to_rdkit()
AllChem.EmbedMultipleConfs(rdmol, numConfs=10, randomSeed=42)

In [4]:
#test the optimizer

optimize_MMFF = MMFFOptimizeMolecule(rdmol,mmffVariant = 'MMFF94',confId=3) #,confId=1,mmffvariant='MMFF94'
optimize_MMFF



0

In [5]:
#test all the conformers

for id in range(rdmol.GetNumConformers()):
    optimize_MMFF = MMFFOptimizeMolecule(rdmol,mmffVariant = 'MMFF94',confId=3) #,confId=1,mmffvariant='MMFF94'
    print(f'conformer:{id}, optimize (0 = converge, 1 = more iterations req): {optimize_MMFF}')

conformer:0, optimize (0 = converge, 1 = more iterations req): 0
conformer:1, optimize (0 = converge, 1 = more iterations req): 0
conformer:2, optimize (0 = converge, 1 = more iterations req): 0
conformer:3, optimize (0 = converge, 1 = more iterations req): 0
conformer:4, optimize (0 = converge, 1 = more iterations req): 0
conformer:5, optimize (0 = converge, 1 = more iterations req): 0
conformer:6, optimize (0 = converge, 1 = more iterations req): 0
conformer:7, optimize (0 = converge, 1 = more iterations req): 0
conformer:8, optimize (0 = converge, 1 = more iterations req): 0
conformer:9, optimize (0 = converge, 1 = more iterations req): 0


In [6]:
rdmol.GetConformer(1)

In [7]:
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
import py3Dmol


In [8]:
#do the conformers look structurally sensible

for id in range(rdmol.GetNumConformers()):
    IPythonConsole.drawMol3D(rdmol,confId=id)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [9]:
optimize_MMFF

0

In [10]:
#now create a series of xyz files so we can run the conformers with ASE, first convert our conformers to xyz format and then write to file

from openff.units import unit

cwd = os.getcwd()

conformers = []



for confs in range(rdmol.GetNumConformers()):
    conformer = np.zeros((rdmol.GetConformer(confs).GetNumAtoms(), 3))
    for atom_index, coordinates in enumerate(rdmol.GetConformer(confs).GetPositions()):
        conformer[atom_index, :] = coordinates
    conformers.append(conformer * unit.angstrom)


for number ,conformer in enumerate(conformers):
    atoms = [
            {
                "element": SYMBOLS[atom.atomic_number],
                "x": conformer[index, 0],
                "y": conformer[index, 1],
                "z": conformer[index, 2],
            }
            for index, atom in enumerate(molecule.atoms)
        ]
    xyz = f'{molecule.n_atoms}\n{molecule.to_smiles()}\n'
    for row in atoms:
        xyz += f"{row['element']}\t{np.around(row['x'].magnitude,decimals=6)}\t{np.around(row['y'].magnitude,decimals=6)}\t{np.around(row['z'].magnitude, decimals=6)}\n"
    try:
        f = open(cwd+f"/conformer_{number}.xyz", 'x')
        f.write(xyz)
        f.close()
    except FileExistsError:
        continue

In [11]:
from ase.calculators.gaussian import Gaussian, GaussianOptimizer
import ase.io
from ase.visualize import view
!module load gaussian/09




/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [12]:
Gaussian.command = '/mnt/storage/apps/eb/software/gaussian/09/g09 < PREFIX.com > PREFIX.log'

In [13]:
test_atoms = ase.io.read('conformer_0.xyz')

In [14]:
view(test_atoms)

<Popen: returncode: None args: ['/mnt/nfs/home/nca121/mambaforge/envs/openff...>

In [15]:
from openff.recharge.utilities.molecule import smiles_to_molecule
from openff.recharge.conformers import ConformerGenerator, ConformerSettings
from openff.units.elements import SYMBOLS
import numpy as np
from openff.units import unit


In [16]:
molecule = smiles_to_molecule("OCC(O)CO")
conformers = ConformerGenerator.generate(
    molecule, ConformerSettings(max_conformers=10)
)

In [17]:
def conf_to_xyz_string(conformer, molecule):
    atoms = [
                    {
                        "element": SYMBOLS[atom.atomic_number],
                        "x": conformer[index, 0],
                        "y": conformer[index, 1],
                        "z": conformer[index, 2],
                    }
                    for index, atom in enumerate(molecule.atoms)
                ]
    xyz = f'{molecule.n_atoms}\n{molecule.to_smiles()}\n'
    for row in atoms:
        xyz += f"{row['element']}\t{np.around(row['x'].magnitude,decimals=6)}\t{np.around(row['y'].magnitude,decimals=6)}\t{np.around(row['z'].magnitude, decimals=6)}\n"
    return xyz

In [18]:
conformers[0]

Magnitude,[[-0.3611847460269928 -0.008062569424510002 0.021483348682522774] [0.9523459672927856 0.5384629368782043 -0.032555222511291504] [1.1439802646636963 1.233094573020935 -1.3743836879730225] [2.4670777320861816 1.7732906341552734 -1.4064496755599976] [0.9855600595474243 0.2572946548461914 -2.534839153289795] [1.1710933446884155 0.9411581158638 -3.7698163986206055] [-0.45401597023010254 -0.4434690475463867 0.8860717415809631] [1.0573564767837524 1.2441487312316895 0.7981725335121155] [1.6660577058792114 -0.28194597363471985 0.09198283404111862] [0.4551912844181061 2.0798325538635254 -1.4719767570495605] [2.5237202644348145 2.3979244232177734 -0.6684095859527588] [-0.009874296374619007 -0.19747895002365112 -2.545755624771118] [1.7366387844085693 -0.5375308394432068 -2.487229347229004] [1.0608800649642944 0.2808915376663208 -4.472022533416748]]
Units,angstrom


In [19]:
type(conformers[0])

openff.units.units.Quantity

In [20]:
from openff.units import unit

cwd = os.getcwd()

conformers = []

molecule = smiles_to_molecule("OCC(O)CO")
rdmol = molecule.to_rdkit()
AllChem.EmbedMultipleConfs(rdmol, numConfs=10, randomSeed=42)

for confs in range(rdmol.GetNumConformers()):
    
    conformer = np.zeros((rdmol.GetConformer(confs).GetNumAtoms(), 3))
    for atom_index, coordinates in enumerate(rdmol.GetConformer(confs).GetPositions()):
        conformer[atom_index, :] = coordinates
    conformers.append(conformer * unit.angstrom)


In [21]:
[molecule.add_conformer(i) for i in conformers]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [22]:
rdmol =  molecule.to_rdkit()

In [23]:
rdmol.GetNumConformers()

10

In [24]:
optimize_MMFF = MMFFOptimizeMolecule(rdmol,mmffVariant = 'MMFF94')

In [25]:
optimized_conformer = Molecule.from_rdkit(rdmol)

In [26]:
optimized_conformer.n_conformers


10

In [27]:
np.zeros(3)

array([0., 0., 0.])

In [28]:
np.array([row.split('\t') for row in xyz_conf.split('\n')[2:]][:-1])

NameError: name 'xyz_conf' is not defined